import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Sampler
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import numpy as np
import random
import math
import time
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# Set random seeds for reproducibility
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


**Submitted by:**

- **Student 1 — Mahmoud Abade, 206773756**
- **Student 2 — Firas Dwere, 214225021**


import tensorflow_datasets as tfds

# Load Portuguese-English dataset
dataset, info = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)
train_examples, val_examples, test_examples = dataset['train'], dataset['validation'], dataset['test']

# Helper to convert TF dataset to list of strings
def tf_to_list(tf_dataset):
    pt_list, en_list = [], []
    for pt, en in tfds.as_numpy(tf_dataset):
        pt_list.append(pt.decode('utf-8'))
        en_list.append(en.decode('utf-8'))
    return pt_list, en_list

train_pt, train_en = tf_to_list(train_examples)
val_pt,   val_en   = tf_to_list(val_examples)
test_pt,  test_en  = tf_to_list(test_examples)

print(f"Train size: {len(train_pt)}")
print(f"Val size:   {len(val_pt)}")
print(f"Test size:  {len(test_pt)}")


## Question 1 — Chatbot Tutorial (35 Points)

In this tutorial, we explore a classical and instructive application of
**recurrent sequence-to-sequence (seq2seq) models**: building a neural
chatbot. Chatbots provide a natural and intuitive testbed for studying
sequence models, as they require processing variable-length input
sequences and generating coherent variable-length outputs, one token at
a time.

Conversational agents are a long-standing and active research topic in
artificial intelligence. Chatbots appear in a wide range of practical
settings, including customer service systems, online helpdesks, and
virtual assistants. Many deployed systems rely on **retrieval-based
models**, which select responses from a predefined set based on the input
query. While such approaches can be effective in narrowly defined
domains, they lack the flexibility required for open-domain
conversation.

Teaching a machine to generate meaningful, context-aware responses
across multiple domains remains a challenging and largely unsolved
problem. The rise of deep learning has enabled a class of **generative
conversational models**, most notably the *Neural Conversational Model*
introduced by Vinyals and Le (2015), which demonstrated that an
encoder–decoder architecture can be trained end-to-end to map input
sentences directly to output sentences.

Inspired by this line of work, we study a simplified generative chatbot
implemented using modern deep learning tools.

In this question, we will work with conversational data extracted from
movie scripts in the
[Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html).
Each training example consists of an input sentence and a corresponding
response sentence, forming a paired sequence-to-sequence learning task.


### 1. Tokenization & Vocabulary

We convert text into numbers for the model.

*   **Tokens**: Words are mapped to unique integers (indices).
*   **Vocabularies**: Built separately for Portuguese (Source) and English (Target).
*   **Special Tokens**:
    *   `<pad>`: Padding for equal batch lengths.
    *   `<bos>`: Beginning of sentence.
    *   `<eos>`: End of sentence.
    *   `<unk>`: Unknown words.


# Special tokens
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"

SPECIAL_TOKENS = [PAD_TOKEN, UNK_TOKEN, BOS_TOKEN, EOS_TOKEN]

def tokenize(sentence: str):
    """Splits sentence into tokens."""
    return sentence.strip().split()

class Vocabulary:
    def __init__(self, sentences, min_freq=2):
        counter = Counter()
        for sent in sentences:
            counter.update(tokenize(sent))

        # Start with special tokens
        self.itos = list(SPECIAL_TOKENS)
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}

        # Add words that appear at least min_freq times
        for word, freq in counter.items():
            if freq >= min_freq and word not in self.stoi:
                self.stoi[word] = len(self.itos)
                self.itos.append(word)

    def encode(self, sentence, add_bos=False, add_eos=False):
        """Converts text to list of indices."""
        tokens = tokenize(sentence)
        ids = [self.stoi.get(tok, self.stoi[UNK_TOKEN]) for tok in tokens]

        if add_bos:
            ids = [self.stoi[BOS_TOKEN]] + ids
        if add_eos:
            ids = ids + [self.stoi[EOS_TOKEN]]

        return ids

    def decode(self, ids):
        """Converts indices back to text."""
        words = []
        for idx in ids:
            word = self.itos[idx]
            if word in {BOS_TOKEN, PAD_TOKEN}:
                continue
            if word == EOS_TOKEN:
                break
            words.append(word)
        return " ".join(words)

    def __len__(self):
        return len(self.itos)

# Build vocabularies
pt_vocab = Vocabulary(train_pt, min_freq=2)
en_vocab = Vocabulary(train_en, min_freq=2)

print("Portuguese vocab size:", len(pt_vocab))
print("English vocab size:", len(en_vocab))


# Check encoding and decoding
pt_example = train_pt[0]
encoded = pt_vocab.encode(pt_example, add_bos=True, add_eos=True)
decoded = pt_vocab.decode(encoded)

print("Original:", pt_example)
print("Encoded:", encoded[:10], "...")
print("Decoded:", decoded)


In [24]:
### 2. Batching

We use **Bucketing** to group sentences of similar lengths together. This minimizes padding and speeds up training.

class TranslationDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab):
        self.src = src_sentences
        self.tgt = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        src_ids = self.src_vocab.encode(self.src[idx])
        tgt_ids = self.tgt_vocab.encode(self.tgt[idx], add_bos=True, add_eos=True)
        return src_ids, tgt_ids

def collate_fn(batch):
    """Pads batch of variable length sequences."""
    src_batch, tgt_batch = zip(*batch)

    src_lens = torch.tensor([len(x) for x in src_batch], dtype=torch.long)
    tgt_lens = torch.tensor([len(x) for x in tgt_batch], dtype=torch.long)

    # Pad sequences
    src_pad = pad_sequence([torch.tensor(x) for x in src_batch], batch_first=True, padding_value=pt_vocab.stoi[PAD_TOKEN])
    tgt_pad = pad_sequence([torch.tensor(x) for x in tgt_batch], batch_first=True, padding_value=en_vocab.stoi[PAD_TOKEN])

    return src_pad, src_lens, tgt_pad, tgt_lens

class BucketBatchSampler(Sampler):
    """Yields batches where examples have similar lengths."""
    def __init__(self, lengths, batch_size, bucket_size=2048, shuffle=True, drop_last=False):
        self.lengths = np.asarray(lengths)
        self.batch_size = batch_size
        self.bucket_size = bucket_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.lengths))
        self.drop_last = drop_last

    def __iter__(self):
        idxs = self.indices.copy()
        if self.shuffle: np.random.shuffle(idxs)

        # Sort inside large buckets
        for i in range(0, len(idxs), self.bucket_size):
            bucket = idxs[i:i + self.bucket_size]
            bucket = bucket[np.argsort(self.lengths[bucket])]

            for j in range(0, len(bucket), self.batch_size):
                batch = bucket[j:j + self.batch_size]
                if self.drop_last and len(batch) < self.batch_size: continue
                yield batch.tolist()

    def __len__(self):
        n = len(self.indices) // self.batch_size
        if not self.drop_last and len(self.indices) % self.batch_size != 0:
            n += 1
        return n

# Prepare DataLoaders
train_dataset = TranslationDataset(train_pt, train_en, pt_vocab, en_vocab)
val_dataset   = TranslationDataset(val_pt,   val_en,   pt_vocab, en_vocab)

train_src_lengths = [len(pt_vocab.encode(s)) for s in train_pt]
val_src_lengths   = [len(pt_vocab.encode(s)) for s in val_pt]

BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_sampler=BucketBatchSampler(train_src_lengths, BATCH_SIZE), collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset,   batch_sampler=BucketBatchSampler(val_src_lengths, BATCH_SIZE, shuffle=False), collate_fn=collate_fn)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches:   {len(val_loader)}")


In [26]:
# Splits each line of the file to create lines and conversations
def loadLinesAndConversations(fileName):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            lineJson = json.loads(line)
            # Extract fields for line object
            lineObj = {}
            lineObj["lineID"] = lineJson["id"]
            lineObj["characterID"] = lineJson["speaker"]
            lineObj["text"] = lineJson["text"]
            lines[lineObj['lineID']] = lineObj

            # Extract fields for conversation object
            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj

    return lines, conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

### 3. Model Architecture (Seq2Seq with Attention)

In [27]:
We implement a **Seq2Seq** model with **Attention**.

1.  **Encoder**: Bidirectional LSTM processes the source sentence.
2.  **Decoder**: Unidirectional LSTM generates the translation step-by-step.
3.  **Attention**: Allows the decoder to focus on relevant source parts.



Processing corpus into lines and conversations...

Writing newly formatted file...

Sample lines from file:
b'They do to!\tThey do not!\n'
b'She okay?\tI hope so.\n'
b"Wow\tLet's go.\n"
b'"I\'m kidding.  You know how sometimes you just become this ""persona""?  And you don\'t know how to quit?"\tNo\n'
b"No\tOkay -- you're gonna need to learn how to lie.\n"
b"I figured you'd get to the good stuff eventually.\tWhat good stuff?\n"
b'What good stuff?\t"The ""real you""."\n'
b'"The ""real you""."\tLike my fear of wearing pastels?\n'
b'do you listen to this crap?\tWhat crap?\n'
b"What crap?\tMe.  This endless ...blonde babble. I'm like, boring myself.\n"


#### Trim Data

#### Encoder
Encodes source tokens into context vectors.

In [28]:
class EncoderBiLSTM(nn.Module):
    def __init__(self, src_vocab_size, emb_dim, hidden_dim, pad_idx, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(src_vocab_size, emb_dim, padding_idx=pad_idx)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True)

    def forward(self, src_pad, src_lens):
        # src_pad: (B, T_src)
        embedded = self.dropout(self.embedding(src_pad))

        # Pack for efficient processing of variable lengths
        packed = pack_padded_sequence(embedded, src_lens.cpu(), batch_first=True, enforce_sorted=False)
        outputs, (hidden, cell) = self.lstm(packed)
        outputs, _ = pad_packed_sequence(outputs, batch_first=True)

        # outputs: (B, T_src, 2H)
        # hidden, cell: (2, B, H)
        return outputs, (hidden, cell)


#### Attention
Calculates a weighted sum of encoder states (context) based on relevance to the current decoder state.

class DotProductAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        # Project encoder states (2H) to match decoder dim (H)
        self.W_p = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, dec_h, enc_hiddens, src_lens):
        # dec_h: (B, H), enc_hiddens: (B, T_src, 2H)

        # Score = dec_h dot W_p(enc_hiddens)
        enc_proj = self.W_p(enc_hiddens) 
        scores = torch.bmm(enc_proj, dec_h.unsqueeze(2)).squeeze(2) # (B, T_src)

        # Mask padding
        max_len = enc_hiddens.size(1)
        mask = torch.arange(max_len, device=scores.device).expand(len(src_lens), max_len) < src_lens.unsqueeze(1)
        scores = scores.masked_fill(~mask, -1e9)

        # Normalize to get weights
        attn_weights = F.softmax(scores, dim=1)

        # Context = Weighted sum of encoder hidden states
        context = torch.bmm(attn_weights.unsqueeze(1), enc_hiddens).squeeze(1) # (B, 2H)

        return context, attn_weights


In [30]:
#### Decoder
Generates the target sequence. We initialize it using the encoder's final state.

keep_words 7833 / 18079 = 0.4333
Trimmed from 64313 pairs to 53131, 0.8261 of total


class DecoderInit(nn.Module):
    """Adapts BiLSTM encoder final states (2H) to UniLSTM decoder init states (H)."""
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_proj = nn.Linear(hidden_dim * 2, hidden_dim)
        self.cell_proj = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, h_n, c_n):
        # Cat forward + backward states
        h_cat = torch.cat((h_n[0], h_n[1]), dim=1)
        c_cat = torch.cat((c_n[0], c_n[1]), dim=1)
        
        # Project
        h0 = torch.tanh(self.hidden_proj(h_cat))
        c0 = torch.tanh(self.cell_proj(c_cat))
        return h0, c0


class Decoder(nn.Module):
    def __init__(self, tgt_vocab_size, emb_dim, hidden_dim, pad_idx, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(tgt_vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTMCell(emb_dim, hidden_dim)
        self.attention = DotProductAttention(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.combine_proj = nn.Linear(hidden_dim + hidden_dim * 2, hidden_dim)

    def forward(self, tgt_pad, dec_init, enc_hiddens, src_lens):
        # tgt_pad: (B, T_tgt)
        batch_size, seq_len = tgt_pad.size()
        h, c = dec_init

        embedded = self.dropout(self.embedding(tgt_pad))
        attn_vecs = []

        # Loop through sequence (Teacher Forcing)
        for t in range(seq_len - 1):
            x_t = embedded[:, t, :]
            h, c = self.lstm(x_t, (h, c))

            # Compute attention context
            context, _ = self.attention(h, enc_hiddens, src_lens)

            # Combine context + hidden
            combined = torch.cat((h, context), dim=1)
            attn_vec = torch.tanh(self.combine_proj(combined))
            attn_vecs.append(attn_vec)

        return torch.stack(attn_vecs, dim=1) # (B, T-1, H)


In [31]:
#### Output Projection
Maps decoder states to vocabulary logits.

class OutputProjection(nn.Module):
    def __init__(self, hidden_dim, tgt_vocab_size):
        super().__init__()
        self.proj = nn.Linear(hidden_dim, tgt_vocab_size)

    def forward(self, attn_vecs):
        return self.proj(attn_vecs)


#### Full Model

In [32]:
class Seq2SeqNMT(nn.Module):
    def __init__(self, encoder, decoder, dec_init, out_proj):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.dec_init = dec_init
        self.out_proj = out_proj

    def forward(self, src_pad, src_lens, tgt_pad):
        enc_hiddens, (h_n, c_n) = self.encoder(src_pad, src_lens)
        dec_init_state = self.dec_init(h_n, c_n)
        attn_vecs = self.decoder(tgt_pad, dec_init_state, enc_hiddens, src_lens)
        logits = self.out_proj(attn_vecs)
        return logits


### 4. Training Loop

# Hyperparameters
EMB_DIM = 128
HIDDEN_DIM = 128
DROPOUT = 0.1
N_EPOCHS = 25
LEARNING_RATE = 0.0005

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Init Model
encoder = EncoderBiLSTM(len(pt_vocab), EMB_DIM, HIDDEN_DIM, pt_vocab.stoi[PAD_TOKEN], DROPOUT)
decoder = Decoder(len(en_vocab), EMB_DIM, HIDDEN_DIM, en_vocab.stoi[PAD_TOKEN], DROPOUT)
dec_init = DecoderInit(HIDDEN_DIM)
out_proj = OutputProjection(HIDDEN_DIM, len(en_vocab))

model = Seq2SeqNMT(encoder, decoder, dec_init, out_proj).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=en_vocab.stoi[PAD_TOKEN])

train_losses, val_losses = [], []

print(f"Training on {device}...")

for epoch in range(N_EPOCHS):
    model.train()
    epoch_loss = 0

    for src_pad, src_lens, tgt_pad, tgt_lens in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
        src_pad, tgt_pad = src_pad.to(device), tgt_pad.to(device)

        optimizer.zero_grad()
        logits = model(src_pad, src_lens, tgt_pad)

        # Targets are shifted by 1
        targets = tgt_pad[:, 1:].reshape(-1)
        logits = logits.reshape(-1, logits.shape[-1])

        loss = criterion(logits, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        epoch_loss += loss.item()

    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    epoch_val_loss = 0
    with torch.no_grad():
        for src_pad, src_lens, tgt_pad, tgt_lens in val_loader:
            src_pad, tgt_pad = src_pad.to(device), tgt_pad.to(device)
            logits = model(src_pad, src_lens, tgt_pad)
            targets = tgt_pad[:, 1:].reshape(-1)
            logits = logits.reshape(-1, logits.shape[-1])
            epoch_val_loss += criterion(logits, targets).item()

    avg_val_loss = epoch_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f}")

# Plot
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Loss Curves')
plt.legend()
plt.show()


### 5. Evaluation (BLEU Score)

In [34]:
def beam_search_decode(model, src_sentence, beam_size=5, max_len=50):
    """Decodes a source sentence using Beam Search."""
    model.eval()
    with torch.no_grad():
        src_ids = pt_vocab.encode(src_sentence)
        src_tensor = torch.tensor([src_ids], dtype=torch.long, device=device)
        src_len = torch.tensor([len(src_ids)], dtype=torch.long)

        # Encode
        enc_hiddens, (h_n, c_n) = model.encoder(src_tensor, src_len)
        h, c = model.dec_init(h_n, c_n)

        # Beam: (score, sequence, (h, c))
        candidates = [(0.0, [en_vocab.stoi[BOS_TOKEN]], (h, c))]
        completed = []

        for _ in range(max_len):
            new_candidates = []
            for score, seq, (curr_h, curr_c) in candidates:
                if seq[-1] == en_vocab.stoi[EOS_TOKEN]:
                    completed.append((score, seq))
                    continue

                inp_token = torch.tensor([seq[-1]], dtype=torch.long, device=device)
                
                # Decoder step
                embed = model.decoder.embedding(inp_token)
                new_h, new_c = model.decoder.lstm(embed, (curr_h, curr_c))
                context, _ = model.decoder.attention(new_h, enc_hiddens, src_len.to(device))
                combined = torch.cat((new_h, context), dim=1)
                attn_vec = torch.tanh(model.decoder.combine_proj(combined))
                log_probs = F.log_softmax(model.out_proj(attn_vec), dim=1)

                # Top k
                topv, topi = log_probs.topk(beam_size)
                for v, i in zip(topv[0], topi[0]):
                    new_candidates.append((score + v.item(), seq + [i.item()], (new_h, new_c)))

            if not new_candidates: break
            
            # Sort and keep top beam_size
            candidates = sorted(new_candidates, key=lambda x: x[0], reverse=True)[:beam_size]

        if not completed: completed = candidates
        best_seq = sorted(completed, key=lambda x: x[0], reverse=True)[0][1]
        return en_vocab.decode(best_seq)

def compute_bleu(references, hypotheses):
    """Computes Corpus-level BLEU-4."""
    precisions = []
    for n in range(1, 5):
        correct, total = 0, 0
        for ref, hyp in zip(references, hypotheses):
            ref_tokens, hyp_tokens = ref.split(), hyp.split()
            ref_counts = Counter([tuple(ref_tokens[i:i+n]) for i in range(len(ref_tokens)-n+1)])
            hyp_counts = Counter([tuple(hyp_tokens[i:i+n]) for i in range(len(hyp_tokens)-n+1)])
            for gram, count in hyp_counts.items():
                total += count
                correct += min(count, ref_counts.get(gram, 0))
        precisions.append(correct / total if total > 0 else 0)

    if min(precisions) == 0: return 0.0
    geo_mean = math.exp(sum(math.log(p) for p in precisions) / 4)
    
    # Brevity Penalty
    ref_len = sum(len(r.split()) for r in references)
    hyp_len = sum(len(h.split()) for h in hypotheses)
    bp = 1.0 if hyp_len > ref_len else math.exp(1 - ref_len / hyp_len) if hyp_len > 0 else 0
    return bp * geo_mean

# Evaluate on Test Set
print("Evaluating...")
hypotheses, references = [], []
for pt, en in tqdm(zip(test_pt, test_en), total=len(test_pt)):
    hypotheses.append(beam_search_decode(model, pt))
    references.append(en)

score = compute_bleu(references, hypotheses)
print(f"BLEU Score: {score*100:.2f}")

# Examples
for i in range(3):
    print(f"Src: {test_pt[i]}\nRef: {test_en[i]}\nHyp: {hypotheses[i]}\n" + "-"*20)


### Key Concepts Reflection

**1. Parallel Corpus**: A dataset of source-target sentence pairs used for supervised translation training.

**2. Special Tokens**:
*   `<pad>`: Handles variable lengths.
*   `<unk>`: Manages out-of-vocabulary words.
*   `<bos>`/`<eos>`: Marks sequence boundaries, critical for the autoregressive decoder to start and stop.

**3. Encoder-Decoder Init**: The encoder (Bidirectional) outputs two states (Forward+Backward, `2H`), while the decoder (Unidirectional) needs one (`H`). We project the concatenated encoder states to the decoder dimension to initialize it.

**4. Attention**: Solves the bottleneck of compressing a sentence into a single vector. It allows the decoder to "look back" at relevant source tokens at every step by computing a weighted average of encoder outputs.

**5. Teacher Forcing**: Feeding ground-truth tokens as input during training to stabilize learning. In inference, the model must feed its own predictions (autoregressive), which can lead to error accumulation (exposure bias).

**6. BLEU**: Measures n-gram overlap between hypothesis and reference. It's computed on the corpus level to be statistically robust. Beam search improves BLEU by exploring multiple decoding paths rather than just the greedy best.
